# Detectron2 Training

## Install requirements

In [ ]:
%pip install -r requirements.txt

In [ ]:
!python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
import sys
import os
import distutils.core
import torch
import detectron2
from roboflow import Roboflow
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer

In [ ]:
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

## Prepare dataset

Replace these lines with your own credentials. For example:

rf = Roboflow(api_key='YOUR_API_KEY')

project = rf.workspace('YOUR_WORKSPACE').project('YOUR_PROJECT')

version = project.version(YOUR_VERSION_NUMBER)

dataset = version.download('YOUR_DATASET_FORMAT')

In [ ]:
rf = Roboflow(api_key="00000000000000000000")
project = rf.workspace("team").project("project")
version = project.version(0)
dataset = version.download("format:coco")

In [ ]:
train_annot = dataset.location + '/train/_annotations.coco.json'
train_img = dataset.location + '/train'
val_annot = dataset.location + '/valid/_annotations.coco.json'
val_img = dataset.location + '/valid'
test_annot = dataset.location + '/test/_annotations.coco.json'
test_img = dataset.location + '/train'

In [ ]:
register_coco_instances("train_dataset", {}, train_annot, train_img)
register_coco_instances("valid_dataset", {}, val_annot, val_img)
register_coco_instances("test_dataset", {}, test_annot, test_img)

## Training

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

### Set arguments

In [ ]:
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2  
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []       
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

In [ ]:
# Start train
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()